# Setting

In [16]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time, re
import requests as rq
from bs4 import BeautifulSoup as bs

In [17]:
# 수집하고자 하는 상품의 URL
# url = 'https://www.lotteon.com/p/product/LM8801056087623?sitmNo=LM8801056087623_001&mall_no=1&dp_infw_cd=SCH%EC%95%84%EC%9D%B4%EC%8B%9C%EC%8A%A4%20%EB%A1%AF%EB%8D%B0&areaCode=SCH'
url = 'https://www.lotteon.com/p/product/LM8801007529202?sitmNo=LM8801007529202_001&mall_no=1&dp_infw_cd=SCH%EA%B9%80%EC%B9%98&areaCode=SCH'

# 수집할 리뷰 개수
n = 100

In [18]:
# 크롬 드라이버 설치
driver_path = ChromeDriverManager().install()
print('크롬 드라이버 설치 경로 : ', driver_path)

크롬 드라이버 설치 경로 :  C:\Users\njh26\.wdm\drivers\chromedriver\win64\122.0.6261.128\chromedriver-win32/chromedriver.exe


In [19]:
# 브라우저 인스턴스 생성, url 오픈
driver = webdriver.Chrome(service=Service(driver_path))
driver.get(url)
driver.implicitly_wait(10)

# 팝업창 닫기
try:
    popupclose = driver.find_element(By.CLASS_NAME, 'popupClose')
    popupclose.click()
except selenium.common.exceptions.NoSuchElementException:
    pass

# Crawling

In [20]:
# 리뷰를 저장할 리스트 생성
reviews = []
ratings = []
days = []
sellers = []
first_keywords = []
second_keywords = []
third_keywords = []

# 크롤링 함수
def crawling():
    # 리뷰
    review = driver.find_elements(By.CLASS_NAME, 'texting')
    for a in review:
        review_text = a.text
        cleaned_text = re.sub(r'[\U0001F300-\U0001F6FF]', '', review_text)  # 이모지 제거 <- 파일 저장시 오류발생
        reviews.append(cleaned_text)

    # 별점
    rating = driver.find_elements(By.CSS_SELECTOR, '#reviewMain > div > div.contents > div.staring > em')
    for a in rating:
        ratings.append(a.text)

    # 리뷰작성일
    day = driver.find_elements(By.CLASS_NAME, 'date')
    for a in day:
        days.append(a.text)

    # 판매자
    seller = driver.find_elements(By.CSS_SELECTOR, '#reviewMain > div > div.contents > div.seller > span > span')
    for a in seller:
        sellers.append(a.text)

    # 키워드 1 
    first_keyword = driver.find_elements(By.CSS_SELECTOR, '#reviewMain > div > div.reviewBox.specification > dl:nth-child(1) > dd')
    for a in first_keyword:
        first_keywords.append(a.text)

    # 키워드 2
    second_keyword = driver.find_elements(By.CSS_SELECTOR, '#reviewMain > div > div.reviewBox.specification > dl:nth-child(2) > dd')
    for a in second_keyword:
        second_keywords.append(a.text)

    # 키워드 3
    third_keyword = driver.find_elements(By.CSS_SELECTOR, '#reviewMain > div > div.reviewBox.specification > dl:nth-child(3) > dd')
    for a in third_keyword:
        third_keywords.append(a.text)

In [21]:
# 페이지 수
pages = int((int(n) - 1) / 5 + 1)   #페이지당 리뷰 5개

# 다음 페이지 버튼
next = driver.find_element(By.XPATH, '//*[@id="reviewMain"]/div[8]/div[1]/div/a[9]')  

# 크롤링
for i in range(1, pages + 1):
    if i == 1:
        crawling()
        print(f'{i} page의 리뷰를 모두 가져왔습니다!')
    else:
        # 다음 페이지로 이동
        next.click()
        time.sleep(1)

        crawling()
        print(f'{i} page의 리뷰를 모두 가져왔습니다!')

# 크롤링 결과
df = pd.DataFrame({'reviews': reviews, 'ratings': ratings, 'days': days,
                   'sellers': sellers, 
                   'first_keywords': first_keywords, 'second_keywords': second_keywords, 'third_keywords': third_keywords})
print(f'\n수집한 데이터는 다음과 같습니다. \n {df.head(4)}')

1 page의 리뷰를 모두 가져왔습니다!
2 page의 리뷰를 모두 가져왔습니다!
3 page의 리뷰를 모두 가져왔습니다!
4 page의 리뷰를 모두 가져왔습니다!
5 page의 리뷰를 모두 가져왔습니다!
6 page의 리뷰를 모두 가져왔습니다!
7 page의 리뷰를 모두 가져왔습니다!
8 page의 리뷰를 모두 가져왔습니다!
9 page의 리뷰를 모두 가져왔습니다!
10 page의 리뷰를 모두 가져왔습니다!
11 page의 리뷰를 모두 가져왔습니다!
12 page의 리뷰를 모두 가져왔습니다!
13 page의 리뷰를 모두 가져왔습니다!
14 page의 리뷰를 모두 가져왔습니다!
15 page의 리뷰를 모두 가져왔습니다!
16 page의 리뷰를 모두 가져왔습니다!
17 page의 리뷰를 모두 가져왔습니다!
18 page의 리뷰를 모두 가져왔습니다!
19 page의 리뷰를 모두 가져왔습니다!
20 page의 리뷰를 모두 가져왔습니다!

수집한 데이터는 다음과 같습니다. 
                               reviews ratings        days   sellers  \
0  맛있어요 맛있어요 맛있어요 맛있어요 맛있어요 맛있어요 맛있어요       5  2024.03.01  롯데마트 이천점   
1                     맛있는 김치 잘 먹었습니다        5  2024.02.13  롯데마트 송도점   
2         잘 먹을께요 감사합니다\n만족해요 계속 사는중이요       5  2024.01.27  롯데마트 이천점   
3       맛있어요 맛있어요 맛있어요 맛있어요 맛있어요 맛있어요       5  2024.01.25  롯데마트 이천점   

  first_keywords second_keywords third_keywords  
0         “맛있어요”          “꼼꼼해요”    “재구매의사 있어요”  
1         “맛있어요”          “꼼꼼해요”    “재구매의사 있어요”  
2

In [22]:
now_page = driver.find_element(By.CSS_SELECTOR, '#reviewMain > div.paginationArea.short > div.paginationWrap > div > a > span.active').text
print(f'크롬창 마지막 페이지수 : {int(now_page)}')
print(f'수집 목표 페이지수 : {pages}')

if (int(now_page) == pages) == False :
    print('Need more time sleep...!')
else :
    pass

크롬창 마지막 페이지수 : 20
수집 목표 페이지수 : 20


페이지가 넘어가지 못한 경우 한 페이지가 2번 이상 수집되어, 마지막 페이지 수가 일치하지 않음  
위 코드 또는 육안을 통해 마지막 페이지 수와 일치하는지 확인 필요  

In [23]:
# 키워드명 가져오기
keyword_name = driver.find_elements(By.CSS_SELECTOR, '#reviewMain > div.reviewList > div.reviewBox.specification > dl > dt')
keyword_name_list = [a.text for a in keyword_name][:3]

# 상품명 가져오기
product_name = driver.find_element(By.CLASS_NAME, 'pd-widget1__product-name').text
product_name = product_name.replace('*', 'X')    # * 가 있는 경우 to_csv에서 오류 발생
### df.to_csv에서 오류가 발생한 경우 위 코드에서 문자를 추가로 변환해야 함

In [24]:
# 드라이버 종료
driver.quit()

In [25]:
df.rename(columns = {'first_keywords':keyword_name_list[0],
                     'second_keywords':keyword_name_list[1],
                     'third_keywords':keyword_name_list[2]}, inplace = True)

df.to_csv(f"{product_name}_{n}.csv", encoding='utf-8-sig')